## Deploy Vgg Ssd Model

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, os.path.abspath("../tfssd"))
from tfutil import endpoints
from finetune.model_saver import SaverVggSsd

### Restore the AzureML workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config(path="./config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### Create Accellerated Container Image

In [ ]:
from azureml.core.model import Model
from azureml.core.image import Image
from azureml.accel import AccelOnnxConverter
from azureml.accel import AccelContainerImage

model_ckpt_dir = r'/datadrive/Dropbox/neal/kroger/azml_ssd_vgg/'
model_name = r'ssdvgg'
model_save_path = os.path.join(model_ckpt_dir, model_name)

# model_save_path should NOT exist prior to saving the model
not os.path.exists(model_save_path)

In [ ]:
with SaverVggSsd(model_ckpt_dir) as saver:
    saver.save_for_deployment(model_save_path)
    

In [ ]:
registered_model = Model.register(workspace = ws,
                              model_path = model_save_path,
                              model_name = model_name)
print("Successfully registered: ", registered_model.name, registered_model.description, registered_model.version, '\n', sep = '\t')

input_tensor = saver.input_name_str
output_tensors_str = ",".join(saver.output_names)

# Convert model
convert_request = AccelOnnxConverter.convert_tf_model(ws, registered_model, input_tensor, output_tensors_str)
# If it fails, you can run wait_for_completion again with show_output=True.
convert_request.wait_for_completion(show_output=True)
converted_model = convert_request.result
print("\nSuccessfully converted: ", converted_model.name, converted_model.url, converted_model.version, 
      converted_model.id, converted_model.created_time, '\n')

In [ ]:
# Package into AccelContainerImage
image_config = AccelContainerImage.image_configuration()
# Image name must be lowercase
image_name = "{}-image".format(model_name)
image = Image.create(name = image_name,
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output=True)
print("Created AccelContainerImage: {} {} {}\n".format(image.name, image.creation_state, image.image_location))

### Deploy to AKS Cluster

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Uses the specific FPGA enabled VM (sku: Standard_PB6s)
# Standard_PB6s are available in: eastus, westus2, westeurope, southeastasia
prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_PB6s",
                                                    agent_count = 1, 
                                                    location = "westus2")

aks_name = 'aks-pb6-obj2'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
aks_target.wait_for_completion(show_output=True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

### Deploy AccelContainerImage to AKS ComputeTarget

In [ ]:
from azureml.core.webservice import Webservice, AksWebservice

# Set the web service configuration (for creating a test service, we don't want autoscale enabled)
# Authentication is enabled by default, but for testing we specify False
aks_config = AksWebservice.deploy_configuration(autoscale_enabled=False,
                                                num_replicas=1,
                                                auth_enabled = False)

aks_service_name ='my-aks-service'

aks_service = Webservice.deploy_from_image(workspace = ws,
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)

In [ ]:
# Using the grpc client in AzureML Accelerated Models SDK
from azureml.accel import PredictionClient

In [ ]:
address = aks_service.scoring_uri
ssl_enabled = address.startswith("https")
address = address[address.find('/')+2:].strip('/')
port = 443 if ssl_enabled else 80
print(f"address={address}, port={port}, ssl={ssl_enabled}, name={aks_service.name}")

In [ ]:

# Initialize AzureML Accelerated Models client
client = PredictionClient(address=address,
                          port=port,
                          use_ssl=ssl_enabled,
                          service_name=aks_service.name)

In [ ]:
from tfutil import visualization
output_tensors = saver.output_names

In [ ]:
import glob

image_dir = r'/datadrive/Dropbox/neal/data/kroger/dataset/test/JPEGImages/'
im_files = glob.glob(os.path.join(image_dir, '*.jpg'))
im_file = im_files[-21]

import azureml.accel._external.ssdvgg_utils as ssdvgg_utils

result = client.score_file(path=im_file, input_name=saver.input_name_str, outputs=output_tensors)
classes, scores, bboxes = ssdvgg_utils.postprocess(result, select_threshold=0.5)

plt.rcParams['figure.figsize'] = 15, 15
img = cv2.imread(im_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
visualization.plt_bboxes(img, classes, scores, bboxes)


In [ ]:
result